In [ ]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

In [ ]:
#最大最小化
df_temp = MinMaxScaler().fit_transform(df)
#標準化
df_temp = StandardScaler().fit_transform(df)

### 去除極端值 Day 20

In [ ]:
# 方法一 將 1stFlrSF 限制在 500 到 2000 以內, 去除極端值
df['1stFlrSF'] = df['1stFlrSF'].clip(500, 2000)

# 方法二  將 1stFlrSF 限制在 500 到 2000 以內, 捨棄離群值
keep_indexs = (df['1stFlrSF']> 500) & (df['1stFlrSF']< 2000)
df = df[keep_indexs]
train_Y = train_Y[keep_indexs]

In [ ]:
# 將 Fare 取 log1p 
df_fixed = copy.deepcopy(df)
df_fixed['Fare'] = np.log1p(df_fixed['Fare'])
#取log前需要加1, 以避免 0 取 log 時無意義的情形
boxcox 也需要先加 1 , 再進行計算即可
df_fixed['Fare'] = df_fixed['Fare'] + 1
df_fixed['Fare'] = stats.boxcox(df_fixed['Fare'])[0]

### Label Encoder and One Hot Encoder Day22

In [ ]:
# 標籤編碼 
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]

# 獨熱編碼
df_temp = pd.get_dummies(df)

### GroupBy_Encoder Day27

In [ ]:
# 生活總面積(GrLivArea)對販售條件(SaleCondition)做群聚編碼
# 寫法類似均值編碼, 只是對另一個特徵, 而非目標值
df['SaleCondition'] = df['SaleCondition'].fillna('None')
mean_df = df.groupby(['SaleCondition'])['GrLivArea'].mean().reset_index()
mode_df = df.groupby(['SaleCondition'])['GrLivArea'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['SaleCondition'])['GrLivArea'].median().reset_index()
max_df = df.groupby(['SaleCondition'])['GrLivArea'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, median_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, max_df, how='left', on=['SaleCondition'])
temp.columns = ['SaleCondition', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']
temp

### 特徵選擇 Day28

* 過濾法(Filter):選定統計數值與設定門檻，刪除低於門檻的特徵
* 包裝法(Wrapper):根據目標函數，逐步加入特徵或刪除特徵
* 嵌入法(Enbedded):使用機器學習模型，根據擬合後的係數，刪除低於門檻的特徵
    * L1(Lasso)嵌入法
    * GDBT(梯度提升樹)嵌入法

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = df.corr()
sns.heatmap(corr)
plt.show()
# 篩選相關係數大於 0.1 或小於 -0.1 的特徵
high_list = list(corr[(corr['SalePrice']>0.1) | (corr['SalePrice']<-0.1)].index)
high_list.pop(-1)
print(high_list)


##L1 Lasso
from sklearn.linear_model import Lasso
L1_Reg = Lasso(alpha=0.001)
train_X = MMEncoder.fit_transform(df)
L1_Reg.fit(train_X, train_Y)
L1_Reg.coef_
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
df.columns[L1_mask]

from itertools import compress
L1_mask = list((L1_Reg.coef_>0) | (L1_Reg.coef_<0))
L1_list = list(compress(list(df), list(L1_mask)))
L1_list


### 特徵篩選

In [ ]:
# 梯度提升樹擬合後, 將結果依照重要性由高到低排序 (note : D27作業中'Ticket'是第一名特徵, 'Age'是數值特徵中排名最高者)
estimator = GradientBoostingClassifier()
estimator.fit(df.values, train_Y)
feats = pd.Series(data=estimator.feature_importances_, index=df.columns)
feats = feats.sort_values(ascending=False)
feats

### 葉編碼 Day30

In [ ]:
#待解答